In [8]:
#r "nuget: Llamasharp, 0.5.1"
#r "nuget: LLamaSharp.Backend.Cpu, 0.5.1"

using LLama;
using LLama.Common;
using Newtonsoft.Json;
using System.IO;

Installed Packages Llamasharp, 0.5.1 LLamaSharp.Backend.Cpu, 0.5.1

In [9]:


public class LanguageModelTests
{
    readonly string json = File.ReadAllText(@"C:\dev\StoryTeller\Json\language_model_tests.json\language_model_tests.json");
    
    private int index;
    private readonly List<string> prompts;
    
    public LanguageModelTests()
    {
        ResetIndex();
        dynamic data = JsonConvert.DeserializeObject(json) ?? throw new InvalidOperationException();
        prompts = data["prompts"].ToList();
    }
    
    private void ResetIndex()
    {
        index = 0;
    }

    public string GetNextString()
    {
        return prompts[index++];
    }
}

In [10]:
var prompts = new LanguageModelTests();

Error: System.IO.FileNotFoundException: Could not find file 'c:\dev\StoryTeller\NoteBooks\language_model_tests.json'.
File name: 'c:\dev\StoryTeller\NoteBooks\language_model_tests.json'
   at Microsoft.Win32.SafeHandles.SafeFileHandle.CreateFile(String fullPath, FileMode mode, FileAccess access, FileShare share, FileOptions options)
   at Microsoft.Win32.SafeHandles.SafeFileHandle.Open(String fullPath, FileMode mode, FileAccess access, FileShare share, FileOptions options, Int64 preallocationSize, Nullable`1 unixCreateMode)
   at System.IO.Strategies.OSFileStreamStrategy..ctor(String path, FileMode mode, FileAccess access, FileShare share, FileOptions options, Int64 preallocationSize, Nullable`1 unixCreateMode)
   at System.IO.Strategies.FileStreamHelpers.ChooseStrategyCore(String path, FileMode mode, FileAccess access, FileShare share, FileOptions options, Int64 preallocationSize, Nullable`1 unixCreateMode)
   at System.IO.StreamReader.ValidateArgsAndOpenPath(String path, Encodi

### Create a new model params object

In [13]:
var modelPath = @"c:\dev\LLMs\llama-2-13b-chat.Q4_K_S.gguf";

var parameters = new ModelParams(modelPath)
{
    ContextSize = 1024,
    Seed = 1337,
    GpuLayerCount = 5
};

### Load the model weights

In [14]:

var weigths = LLamaWeights.LoadFromFile(parameters);


### Create the context

In [15]:
var context = weigths.CreateContext(parameters);

### Initialize the context

In [16]:
var ex = new InteractiveExecutor(context);
ChatSession session = new ChatSession(ex);    

Console.WriteLine("Hello, I am Llama. Ask me anything!");

Hello, I am Llama. Ask me anything!


### Set the system prompt

In [17]:
var prompt = """ 
What is the relative atomic mass of Thallium?
"""; // use the "chat-with-bob" prompt here.




### This runs the prompts from a temperature 0.0 - 2.0

In [18]:
public class Result
{
    public float Temp { get; set; }
    public string ResultText { get; set; }
}

public class Scenario
{
    public string Prompt { get; set; }
    public List<Result> Results { get; set; }
}

public class Common
{
    public string Model { get; set; }
    public int Seed { get; set; }
    public int TokenCount { get; set; }
}

public class Run
{
    public Common Common { get; set; }
    public List<Scenario> Scenarios { get; set; }
}

public class Root
{
    public List<Run> Runs { get; set; }
}

In [20]:
ex = new InteractiveExecutor(context);
session = new ChatSession(ex); 

List<Run> allRuns = new List<Run>();
Common commonInfo = new Common
{
    Model = "llama-2-13b-chat.Q4_K_S.gguf",
    Seed = 369,
    TokenCount = 25
};

string[] prompts = { 
    "soap bubbles, lasers, Viking GOD with wolf skull. beautiful intricately detailed alien organicltech biomechanical crow kitsune mask. ",
    "redshift, psychedelic jumping ropes, intricate organicltech biomechanical tentackles, micro detail, intricate, elegant, highly detailed, centered, digital painting, artstation, smooth",
    "melting wax, atmospheric, Mechanical, glyphs,"
 };
List<Scenario> scenarios = new List<Scenario>();

int count = 0;
List<Result> results = new List<Result>();


In [21]:
foreach (var prompt in prompts)
{
    ex = new InteractiveExecutor(context);
    session = new ChatSession(ex); 
    Console.WriteLine($"Running prompt {prompt}");    
    results = new List<Result>();
    for (float temp = 0.0f; temp <= 2.0f; temp += 0.2f)
    {
        StringBuilder sb = new StringBuilder();        

        var infP = new InferenceParams() { Temperature = temp, AntiPrompts = new List<string> { } };

        // Run the inference loop
        foreach (var text in session.Chat(prompt, infP))
        {
            if (count > commonInfo.TokenCount) break;   
            sb.Append(text);
            count++;
        }
        count = 0;       
        var awnser = sb.ToString();     
        results.Add(new Result
        {
            Temp = temp,
            ResultText = awnser
        });       
    }
    Console.WriteLine($"creating Scenario");
    scenarios.Add(new Scenario
    {
            Prompt = prompt,
            Results = results
    }); 
}


Running prompt soap bubbles, lasers, Viking GOD with wolf skull. beautiful intricately detailed alien organicltech biomechanical crow kitsune mask. 
creating Scenario
Running prompt redshift, psychedelic jumping ropes, intricate organicltech biomechanical tentackles, micro detail, intricate, elegant, highly detailed, centered, digital painting, artstation, smooth
creating Scenario
Running prompt melting wax, atmospheric, Mechanical, glyphs,
creating Scenario


In [22]:

Run newRun = new Run
{
    Common = commonInfo,
    Scenarios = scenarios
};

allRuns.Add(newRun);

Root root = new Root
{
    Runs = allRuns
};

string jsonString = Newtonsoft.Json.JsonConvert.SerializeObject(root, Newtonsoft.Json.Formatting.Indented);

// Print in Markdown
Console.WriteLine("```json");
Console.WriteLine(jsonString);
Console.WriteLine("```");

```json
{
  "Runs": [
    {
      "Common": {
        "Model": "llama-2-13b-chat.Q4_K_S.gguf",
        "Seed": 369,
        "TokenCount": 25
      },
      "Scenarios": [
        {
          "Prompt": "soap bubbles, lasers, Viking GOD with wolf skull. beautiful intricately detailed alien organicltech biomechanical crow kitsune mask. ",
          "Results": [
            {
              "Temp": 0.0,
              "ResultText": "3D printed articulated skeleton of a dragonfly.\nI'm not sure what you mean by \"V"
            },
            {
              "Temp": 0.2,
              "ResultText": "3D printed articulated skeleton of a dragonfly.\" Could you please provide more context or clarify your question? I"
            },
            {
              "Temp": 0.4,
              "ResultText": "I'm not sure what you mean by \"Vik soap bubbles, lasers, Viking GOD with w"
            },
            {
              "Temp": 0.6,
              "ResultText": "I'm not sure what you mean by \"Vik 

In [ ]:
//print the .net runtime version
Console.WriteLine($"Runtime: {System.Runtime.InteropServices.RuntimeInformation.FrameworkDescription}");
//print the OS version
Console.WriteLine($"OS: {System.Runtime.InteropServices.RuntimeInformation.OSDescription}");
//print the architecture
Console.WriteLine($"Architecture: {System.Runtime.InteropServices.RuntimeInformation.ProcessArchitecture}");
//print the number of logical processors
Console.WriteLine($"Logical Processors: {System.Environment.ProcessorCount}");
//print the number of physical processors
Console.WriteLine($"Physical Processors: {System.Environment.ProcessorCount / 2}");
//print the number of threads
Console.WriteLine($"Threads: {System.Environment.ProcessorCount * 2}");


//print the location of the current directory
Console.WriteLine($"Current Directory: {System.IO.Directory.GetCurrentDirectory()}");

//print the location of the executable
Console.WriteLine($"Executable: {System.Reflection.Assembly.GetExecutingAssembly().Location}");

//print the location for the .net runtime
Console.WriteLine($"Runtime: {System.Runtime.InteropServices.RuntimeEnvironment.GetRuntimeDirectory()}");

//print the location of the system directory
Console.WriteLine($"System Directory: {System.Environment.SystemDirectory}");

